In [1]:
from google.colab import drive
import cv2
import numpy as np  
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
drive.mount('/content/gdrive')
import os
import pickle
from sklearn.cluster import MiniBatchKMeans
from google.colab.patches import cv2_imshow

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## **Setting up data paths**

In [0]:
# Root path for the animal images
root_path = "gdrive/My Drive/Computer Vision/Coursework/JPEGImages/" 
# Listing all the elements within the root path images
animal_images = os.listdir(root_path)
# Setting the training and testing data up
training_labels = ['antelope','grizzly+bear','killer+whale','beaver','dalmatian','horse','german+shepherd','blue+whale','siamese+cat','skunk','mole','tiger','moose'
                    ,'spider+monkey','elephant','gorilla','ox','fox','sheep','hamster','squirrel','rhinoceros','rabbit','bat','giraffe','wolf','chihuahua','weasel'
                    ,'otter','buffalo','zebra','deer','bobcat','lion','mouse','polar+bear','collie','walrus','cow','dolphin']
training_data = {}

test_classes = ['chimpanzee','giant+panda','leopard','persian+cat','pig','hippopotamus','humpback+whale','raccoon','rat','seal']

testing_data = {}

## **Loading the training animal images**

In [0]:
for i,classe in enumerate(training_labels):
  print('Processing class: ',i,classe)
  class_paths = os.listdir(root_path+classe)
  orb_dict3[classe] = []
  class_paths_subsample = class_paths[int(len(class_paths))-int(len(class_paths)/3):len(class_paths)]
  for file in class_paths_subsample:
    if file != '.DS_Store':
      img = cv2.imread(root_path+classe+ "/" +file)
      img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      img_resized = cv2.resize(img_RGB,None,fx=0.5,fy=0.5)
      orb_dict3[classe].append(img_resized)

## **Loading the test animal images**

In [0]:
for i,classe in enumerate(test_classes):
  print('Processing class: ',i,classe)
  class_paths = os.listdir(root_path+classe)
  testing_data[classe] = []
  #class_paths_subsample = class_paths[int(len(class_paths))-int(len(class_paths)/3):len(class_paths)]
  for file in class_paths:
    if file != '.DS_Store':
      img = cv2.imread(root_path+classe+ "/" +file)
      img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      img_resized = cv2.resize(img_RGB,None,fx=0.5,fy=0.5)
      testing_data[classe].append(img_resized)

Processing class:  0 chimpanzee
Processing class:  1 giant+panda
Processing class:  2 leopard
Processing class:  3 persian+cat
Processing class:  4 pig
Processing class:  5 hippopotamus
Processing class:  6 humpback+whale
Processing class:  7 raccoon
Processing class:  8 rat
Processing class:  9 seal


In [0]:
#with open('orb_dict1.pickle', 'wb') as handle:
    #pickle.dump(orb_dict1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('gdrive/My Drive/Computer Vision/Coursework/ORB/orb_dict3.pickle', 'rb') as handle1:
   orb_dict3 = pickle.load(handle1)

In [0]:
#combining all the 1/3 pickles to 1 big dictionary
orb_train_dict = orb_dict1.copy()
for feature in [orb_dict2,orb_dict3]:
  for k in feature.keys():
    for v in feature[k]:
      orb_train_dict[k].append(v)

In [0]:
with open('orb_train_dict.pickle', 'wb') as handle:
    pickle.dump(orb_train_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#with open('gdrive/My Drive/Computer Vision/Coursework/ORB/orb_train_dict.pickle', 'rb') as handle1:
   #orb_train_dict = pickle.load(handle1)

In [0]:
def extract_features(orb_image_dict,animal_labels,vector_size=32):
  orb_features = {}
  alg = cv2.KAZE_create()
  for i,animal in enumerate(animal_labels):
    dsc_list = []
    print("Extracting kaze features from: ", i, animal)
    for img in orb_image_dict[animal]:
      kps = alg.detect(img)
      kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
      kps, dsc = alg.compute(img, kps)
      dsc = dsc.flatten()
      dsc_list.append(dsc)
      needed_size = (vector_size * 64)
      if dsc.size < needed_size:
        #if we have less the 32 descriptors then just adding zeros at the
        #end of our feature vector
        dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)]) 
    orb_features[animal] = dsc_list
  return orb_features

In [0]:
kaze_train_features = extract_features(orb_train_dict,training_labels)

In [0]:
kaze_test_features = extract_features(testing_data,test_classes)

In [0]:
#with open('kaze_test_features.pickle', 'wb') as handle:
    #pickle.dump(kaze_test_features, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('gdrive/My Drive/Computer Vision/Coursework/KAZE/kaze_test_features.pickle', 'rb') as handle1:
   kaze_test_features = pickle.load(handle1)

In [0]:
orb_test_features = extract_features(orb_dict,test_classes)

In [0]:
#with open('orb_train_features.pickle', 'wb') as handle:
    #pickle.dump(orb_train_features, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('gdrive/My Drive/Computer Vision/Coursework/KAZE/kaze_train_features.pickle', 'rb') as handle1:
   kaze_train_features = pickle.load(handle1)

In [0]:
#training class
train_class = np.loadtxt('trainclasses.txt', delimiter='\t', usecols=[0], dtype=np.str)  

#testing class
test_class = np.loadtxt('testclasses.txt', delimiter='\t',usecols=[0], dtype=np.str) 

# Reading the predicate binary text file (correlation between animal classes and its attributes)
M = np.loadtxt('predicate-matrix-binary.txt')

# Predicates within the predicate matrix. 85
predicates = np.loadtxt('predicates.txt',delimiter='\t',usecols=[1], dtype=np.str)

#class
classes = np.loadtxt('classes.txt', delimiter='\t', usecols=[1], dtype=np.str)  

In [41]:
print(predicates)

['black' 'white' 'blue' 'brown' 'gray' 'orange' 'red' 'yellow' 'patches'
 'spots' 'stripes' 'furry' 'hairless' 'toughskin' 'big' 'small' 'bulbous'
 'lean' 'flippers' 'hands' 'hooves' 'pads' 'paws' 'longleg' 'longneck'
 'tail' 'chewteeth' 'meatteeth' 'buckteeth' 'strainteeth' 'horns' 'claws'
 'tusks' 'smelly' 'flys' 'hops' 'swims' 'tunnels' 'walks' 'fast' 'slow'
 'strong' 'weak' 'muscle' 'bipedal' 'quadrapedal' 'active' 'inactive'
 'nocturnal' 'hibernate' 'agility' 'fish' 'meat' 'plankton' 'vegetation'
 'insects' 'forager' 'grazer' 'hunter' 'scavenger' 'skimmer' 'stalker'
 'newworld' 'oldworld' 'arctic' 'coastal' 'desert' 'bush' 'plains'
 'forest' 'fields' 'jungle' 'mountains' 'ocean' 'ground' 'water' 'tree'
 'cave' 'fierce' 'timid' 'smart' 'group' 'solitary' 'nestspot' 'domestic']


In [0]:
new_M_matrix = np.delete(M, (24,38,14,5,41,13,17,33,23,34), axis = 0)

In [0]:
#preparing the X Value //training
black_list = []
for animal in train_class:
  for feature in kaze_train_features[animal]:
    black_list.append(feature.ravel())
feature_matrix = np.array(black_list)

In [25]:
(feature_matrix)

array([[ 1.0377282e-02, -2.7707106e-02,  9.7728752e-02, ...,
         1.0015958e-02,  5.9550084e-02,  2.7646190e-02],
       [ 6.8489061e-04, -2.3811727e-03,  4.5076393e-02, ...,
         2.0871963e-02,  6.4833976e-02,  5.2330460e-02],
       [-1.1468822e-02, -1.6773557e-02,  2.0489164e-02, ...,
         2.6638178e-02,  5.3928822e-02,  7.0401542e-02],
       ...,
       [ 3.2011986e-02,  5.4198381e-02,  6.9107287e-02, ...,
         6.7248061e-02,  2.4781395e-02,  6.7273319e-02],
       [-2.2639628e-03, -5.2653000e-02,  5.8395766e-02, ...,
         2.1778759e-02,  1.3170859e-01,  1.0538381e-01],
       [-1.6900769e-01, -1.5192108e-01,  1.6966869e-01, ...,
         1.2277416e-04,  1.7018221e-02,  5.3344887e-02]], dtype=float32)

In [0]:
#minus testing classes
train_pred_dict = {}
column_list = []
for i in range(0,85):
  column_list.append(new_M_matrix[:,i])
  train_pred_dict[predicates[i]] = column_list[i]

In [0]:
#all classes
pred_map = []
pred_dict = {}
for i in range(0,85):
  pred_map.append(M[:,i])
  pred_dict[predicates[i]] = pred_map[i]

In [0]:
#primary imports
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

# Data split for pre-processing
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV

In [0]:
#removing dimensionality that doesn't conform to the matrix. (training)
for animal,features in kaze_train_features.items():
  for feature in features:
    if feature.ravel().shape[0] != 2048:
      features.remove(feature)

In [0]:
#removing dimensionality that doesn't conform to the matrix.
new_kaze_train_features = {}
for animal,features in kaze_train_features.items():
  for feature in features:
    if feature.ravel().shape[0] != 2048:
      print(animal,feature.shape)

In [18]:
#removing dimensionality that doesn't conform to the matrix. (testing)
for animal,features in kaze_test_features.items():
  for feature in features:
    if feature.ravel().shape[0] != 2048:
      features.remove(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


In [0]:
#removing dimensionality that doesn't conform to the matrix.
new_kaze_train_features = {}
for animal,features in kaze_test_features.items():
  for feature in features:
    if feature.ravel().shape[0] != 2048:
      print(animal,feature.shape)

In [0]:
def train_attribute_models(features):
  models = {}
  for att in predicates: 
    y = []
    att_output = train_pred_dict[att]
    for i,animal in enumerate(training_labels):
      if att_output[i] == 1:
        positive = np.ones((len(kaze_train_features[animal]),1), dtype=int)
        for pos_val in positive:
          y.append(*pos_val)
      else:
        negative = np.zeros((len(kaze_train_features[animal]),1), dtype=int)
        for neg_val in negative:
          y.append(*neg_val)
    y = np.array(y)
    X = features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.44, random_state = 42)
    svm = LinearSVC()
    clf = CalibratedClassifierCV(svm)
    clf.fit(X_train,y_train)
    print(att,"classifier trained")
    score = clf.score(X_test,y_test)
    print("The",att,"classifier score is : ",score*100)
    models[att] = clf
  return models

In [0]:
kaze_classifiers1 = train_attribute_models(feature_matrix)

In [0]:
with open('kazeclf1.pickle', 'wb') as handle:
    pickle.dump(kaze_classifiers1, handle, protocol=pickle.HIGHEST_PROTOCOL)
#with open('gdrive/My Drive/Computer Vision/Coursework/KAZE/kazeclf.pickle', 'rb') as handle1:
   #kazeclf = pickle.load(handle1)

In [0]:
def compute_attribute_probs(classifiers):
  prob_dict = {}
  for att,clf in classifiers.items():
    prob_list = []
    for animal,dsc_list in kaze_test_features.items():
      for dsc in dsc_list:
        y_proba = clf.predict_proba([dsc])
        prob_list.append(y_proba)
    prob_dict[att] = prob_list
  return prob_dict

In [0]:
att_prob = compute_attribute_probs(kaze_classifiers1)

In [0]:
with open('att_prob1.pickle', 'wb') as handle:
    pickle.dump(att_prob, handle, protocol=pickle.HIGHEST_PROTOCOL)
#with open('gdrive/My Drive/Computer Vision/Coursework/ORB/kazeclf.pickle', 'rb') as handle1:
   #orb_train_features = pickle.load(handle1)

In [0]:
def compute_attribute_probs(classifiers):
  prob_dict = {}
  for att,clf in classifiers.items():
    prob_list = []
    for animal,dsc_list in kaze_test_features.items():
      for dsc in dsc_list:
        y_proba = clf.predict_proba([dsc])
        prob_list.append(y_proba)
    prob_dict[att] = prob_list
  return prob_dict

In [0]:
probs = compute_attribute_probs(kaze_classifiers1)

In [0]:
#dictionary which contains the key as the attributes and the value as the animal
#outputs whether the attribute is present or not (1 or 0)
predicate_dict = {}
for attribute, predicates in pred_dict.items():
  predicate_dict[attribute]= {}
  for i,animal in enumerate(classes):
    predicate_dict[attribute][animal] = pred_dict[attribute][i]

In [0]:
#method will return a dictonary. key:test_animal_class, value: probabilities of the class being in each img
def compute_class_probs(test_features):
  #dictonary containing as key the test_animal_class, value as the probabilities of each class in img.
  prob_dict = {}
  #total probability multiplication
  total_prob_mult = 1
  for animal in test_features.keys():
    total_prob_mult = 1
    prob_list = []
    for i in range(2455):
      total_prob_mult = 1
      for att in predicates:
        if predicate_dict[att][animal] == 1:
          total_prob_mult = total_prob_mult * probs[att][i][0][1]
        else:
          total_prob_mult = total_prob_mult * probs[att][i][0][0]
      prob_list.append(total_prob_mult)
    prob_dict[animal] = prob_list
  return prob_dict

In [0]:
pre_prob = compute_class_probs(kaze_test_features)

In [0]:
chimpanzee_list = ['chimpanzee'] * 200
giantpanda_list = ['giant+panda'] * 250
leopard_list = ['leopard'] * 200
persiancat_list = ['persian+cat'] * 198
pig_list = ['pig'] * 269 
hippopotamus_list = ['hippopotamus'] * 300
humpbackwhale_list = ['humpback+whale'] * 287
raccoon_list = ['raccoon'] * 200
rat_list = ['rat'] * 251
seal_list = ['seal'] * 300
animal_list = []
animal_list.extend(chimpanzee_list)
animal_list.extend(giantpanda_list)
animal_list.extend(leopard_list)
animal_list.extend(persiancat_list)
animal_list.extend(pig_list)
animal_list.extend(hippopotamus_list)
animal_list.extend(humpbackwhale_list)
animal_list.extend(raccoon_list)
animal_list.extend(rat_list)
animal_list.extend(seal_list)

In [0]:
# outputs a single real number denoting the overall accuracy of your system, averaged
# over the Ntest test images.
def compute_accuracy(test_classes):
  prob_dict = {}
  max_dict = {}
  counter = 0
  for i in range(2455):
    prob_dict = {}
    for animal in test_classes:
      prob_dict[animal] = pre_prob[animal][i]
    if max(prob_dict, key = lambda k: prob_dict[k]) == animal_list[i]:
      counter += 1
  result = (counter/2455) * 100
  return result

In [0]:
accuracy = compute_accuracy(pre_prob)

In [46]:
accuracy

10.386965376782078